In [1]:
# Importing the libraries
import pandas as pd
import numpy as np

from math import nan
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from scikit_posthocs import posthoc_nemenyi_friedman

import shap
from scipy import stats

In [2]:
# load data and see how it looks like
train = pd.read_csv('train', index_col = 0)
train

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,LID_KB,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,LET_IS
1414,60.0,0,0.0,5.0,2.0,1.0,2.0,0.0,6.0,0.0,...,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
227,57.0,0,1.0,1.0,2.0,2.0,2.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
280,52.0,1,1.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0
1538,73.0,0,1.0,6.0,3.0,NaN,2.0,0.0,6.0,4.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
1418,78.0,0,0.0,2.0,2.0,2.0,2.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,55.0,1,3.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0
238,73.0,1,1.0,4.0,2.0,2.0,2.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
1444,65.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
409,75.0,0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0


In [3]:
# Load data and see how it looks like
test = pd.read_csv('test', index_col = 0)
test

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,LID_KB,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,LET_IS
455,50.0,1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
1499,64.0,0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1
1045,61.0,1,1.0,5.0,2.0,1.0,2.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
187,75.0,0,0.0,6.0,1.0,2.0,2.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
1472,62.0,0,0.0,6.0,2.0,1.0,3.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,51.0,1,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
207,76.0,0,0.0,1.0,2.0,2.0,2.0,0.0,7.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
779,83.0,0,1.0,2.0,2.0,1.0,3.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
954,76.0,1,0.0,4.0,2.0,2.0,2.0,0.0,NaN,2.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [4]:
# separate x_train and y_train from train
X_train = train.loc[:, train.columns!='LET_IS']
y_train = pd.Series(train['LET_IS'])

# separate x_test and y_test from test
X_test = test.loc[:, test.columns!='LET_IS']
y_test = pd.Series(test['LET_IS'])

# check the first class is 0 (alive), and the second class is 1 (dead)
y_train.value_counts()
y_test.value_counts()

0    286
1     54
Name: LET_IS, dtype: int64

In [5]:
# XGBoost, our best model
the_bestmodel = xgb.XGBClassifier(
    base_score=None, 
    booster=None, 
    colsample_bylevel=None,
    colsample_bynode=None,
    colsample_bytree=0.9,
    enable_categorical=False, 
    gamma=0.4, 
    gpu_id=None,
    importance_type=None, 
    interaction_constraints=None,
    learning_rate=None,
    max_delta_step=None, 
    max_depth=7,
    min_child_weight=3, 
    missing=nan, 
    monotone_constraints=None,
    n_estimators=100, 
    n_jobs=None, 
    num_parallel_tree=None,
    predictor=None, 
    random_state=1, 
    reg_alpha=1e-05,
    reg_lambda=None, 
    scale_pos_weight=None, 
    subsample=0.7,
    tree_method=None, 
    use_label_encoder=False,
    validate_parameters=None,
    verbosity=None
)

In [6]:
the_bestmodel.fit(X_train,y_train)
yhat = the_bestmodel.predict(X_test)
f1score = f1_score(yhat,y_test)
print(f1score)
print(X_test)

df = pd.DataFrame(data = X_test, columns = ['AGE', 'SEX',
                                            'INF_ANAM', 'STENOK_AN',
                                            'FK_STENOK', 'IBS_POST',
                                            'GB', 'SIM_GIPERT', 'DLIT_AG', 
                                            'ZSN_A', 'nr_11', 'nr_01', 'nr_02',
                                            'nr_03', 'nr_04', 'nr_07', 'nr_08',
                                            'np_01', 'np_04', 'np_05', 'np_07',
                                            'np_08', 'np_09', 'np_10', 'endocr_01',
                                            'endocr_02', 'endocr_03', 'zab_leg_01',
                                            'zab_leg_02', 'zab_leg_03', 'zab_leg_04',
                                            'zab_leg_06', 'S_AD_ORIT', 'D_AD_ORIT',
                                            'O_L_POST', 'K_SH_POST', 'MP_TP_POST', 
                                            'SVT_POST', 'GT_POST', 'FIB_G_POST',
                                            'ant_im', 'lat_im', 'inf_im', 'post_im',
                                            'IM_PG_P', 'ritm_ecg_p_01', 'ritm_ecg_p_02',
                                            'ritm_ecg_p_04', 'ritm_ecg_p_06', 'ritm_ecg_p_07',
                                            'ritm_ecg_p_08', 'n_r_ecg_p_01', 'n_r_ecg_p_02',
                                            'n_r_ecg_p_03', 'n_r_ecg_p_04', 'n_r_ecg_p_05',
                                            'n_r_ecg_p_06', 'n_r_ecg_p_08', 'n_r_ecg_p_09', 
                                            'n_r_ecg_p_10', 'n_p_ecg_p_01', 'n_p_ecg_p_03',
                                            'n_p_ecg_p_04', 'n_p_ecg_p_05', 'n_p_ecg_p_06',
                                            'n_p_ecg_p_07', 'n_p_ecg_p_08', 'n_p_ecg_p_09',
                                            'n_p_ecg_p_10', 'n_p_ecg_p_11', 'n_p_ecg_p_12',
                                            'fibr_ter_01', 'fibr_ter_02', 'fibr_ter_03',
                                            'fibr_ter_05', 'fibr_ter_06', 'fibr_ter_07',
                                            'fibr_ter_08', 'GIPO_K', 'K_BLOOD',
                                            'GIPER_NA', 'NA_BLOOD', 'ALT_BLOOD',
                                            'AST_BLOOD', 'L_BLOOD', 'ROE', 'TIME_B_S',
                                            'NA_KB',
                                            'NOT_NA_KB', 
                                            'LID_KB', 'NITR_S',
                                            'LID_S_n', 'B_BLOK_S_n',
                                            'ANT_CA_S_n', 'GEPAR_S_n',
                                            'ASP_S_n', 'TIKL_S_n', 'TRENT_S_n'])

df

[21:25:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6222222222222222
       AGE  SEX  INF_ANAM  STENOK_AN  FK_STENOK  IBS_POST   GB  SIM_GIPERT  \
455   50.0    1       0.0        0.0        0.0       2.0  0.0         0.0   
1499  64.0    0       0.0        5.0        3.0       2.0  0.0         0.0   
1045  61.0    1       1.0        5.0        2.0       1.0  2.0         0.0   
187   75.0    0       0.0        6.0        1.0       2.0  2.0         0.0   
1472  62.0    0       0.0        6.0        2.0       1.0  3.0         0.0   
...    ...  ...       ...        ...        ...       ...  ...         ...   
749   51.0    1       0.0        1.0        2.0       1.0  0.0         0.0   
207   76.0    0       0.0        1.0        2.0

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,NOT_NA_KB,LID_KB,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
455,50.0,1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
1499,64.0,0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1045,61.0,1,1.0,5.0,2.0,1.0,2.0,0.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
187,75.0,0,0.0,6.0,1.0,2.0,2.0,0.0,7.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1472,62.0,0,0.0,6.0,2.0,1.0,3.0,0.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,51.0,1,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
207,76.0,0,0.0,1.0,2.0,2.0,2.0,0.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
779,83.0,0,1.0,2.0,2.0,1.0,3.0,0.0,NaN,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
954,76.0,1,0.0,4.0,2.0,2.0,2.0,0.0,NaN,2.0,...,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
# Alive result

# force_plot() takes three values: the base value (explainer.expected_value), the SHAP values (shap_values) and the matrix of 
# feature values (X_test)
# The base value (expected value) which is around 0.84, is the average target value of the model output over the training data (X_train)
# The output value, which is 1, is the prediction for that observation (the prediction of the first row in X_test)
# Red/blue: Features that push the prediction higher (to the right) are shown in red, and those pushing the prediction lower are in blue.
# From the result, for example, we can see that the TIME_B_S and AGE are both red. They both have a positive impact on pushing 
# the model higher.  
# However, S_AD_ORIT has the biggest impact compared with other features.

shap.initjs()
explainer = shap.KernelExplainer(the_bestmodel.predict_proba,X_train)
shap_values = explainer.shap_values(df.iloc[0,:])
shap.force_plot(explainer.expected_value[0], shap_values[0], df.iloc[0,:])


Using 1360 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [15]:
# print(explainer.expected_value[0])
# print(shap_values[0])
# print(the_bestmodel.predict_proba())

In [18]:
# Alive result

# The global force plot allows us to see shap values made-up by record. X shows each record, and positive shap values show in red
# and negative shap values are blue.
# Here I chose the X axis as the sample order by output value, and the y axis as the prediction (f(x))
# When I moved the X axis to 1 and click the diagram, there's a popup message saying 'This original index of the sample you clicked is 94'
# This sample has more red contribution than the blue ones, the prediction for this record would be higher than the expected value.

shap_values = explainer.shap_values(df)
shap.force_plot(explainer.expected_value[0], shap_values[0], df)

  0%|          | 0/340 [00:00<?, ?it/s]

In [19]:
# Dead result

# As the description before. Shap values closer to zero means the feature contributes little to the prediction. 
# The base value (expected value) which is around 0.16, is the average target value of the model output over the training data (X_train)
# The output value, which is 0, is the prediction for that observation (the prediction of the first row in X_test)
# Red/blue: Features that push the prediction higher (to the right) are shown in red, and those pushing the prediction lower are in blue.
# From the result, only ant_im (ECG changes in leads V1–V4) has the positive impact, and other features push the prediction lower.
# It is interesting to observe that compared with the alive result, most results are in the opposite direction, 
# like S_AD_ORIT (Systolic blood pressure according to intensive care unit).

shap.initjs()
explainer = shap.KernelExplainer(the_bestmodel.predict_proba,X_train)
shap_values = explainer.shap_values(df.iloc[0,:])
shap.force_plot(explainer.expected_value[1], shap_values[1], df.iloc[0,:])

Using 1360 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [20]:
# The second  class (1, dead)

# When I moved the X axis to 1 and click the diagram, there's a popup message saying 'This original index of the sample you clicked is 117'
# This sample has more positive red contributions than blue ones, the prediction for this record would be higher than the expected value.
# The graph looks very different from the previous class result.

shap_values = explainer.shap_values(df)
shap.force_plot(explainer.expected_value[1], shap_values[1], df)

  0%|          | 0/340 [00:00<?, ?it/s]

## Group project step 9
## Discussion and Limitations: to comment on your results and to enumerate the limitations and future work.

1. Different ways to do feature selection will also change the result of feature selection. However, we've done our best to do feature selection.
2. XGBoost got the best result, but it took a long time.
3. It would be better to compare the SHAP result using the data that only includes features in feature selection. Because it is not easy to see the features impact on global force, I think it is because there are too many features.
4. It seems the SHAP result didn't save, so we need to rerun the code every time. It takes more time to finish and confirm the result.
5. The change of the XGBoost hyperparameter may still change the result of SHAP.
6. We didn't have time to compare the result using feature selection results in the dataset. The performance for the classifier and SHAP must change if using that. Therefore, it is the future work.
7. Everyone tried to do their best to finish the project!